Using: https://medium.com/future-vision/google-maps-in-python-part-2-393f96196eaf

In [1]:
import os
import pandas as pd
import gmaps
from datetime import datetime
import googlemaps

In [2]:
# create function to read in api key from txt file
def read_in_gmaps_api_key(file_directory):
    api_key = pd.read_csv("gmaps_api_key.txt", header = None).iloc[0, 0]
    return api_key

In [3]:
gmaps_api_key = read_in_gmaps_api_key(os.getcwd())

In [15]:
gmaps.configure(api_key=gmaps_api_key)

In [4]:
googlemaps_client = googlemaps.Client(key = gmaps_api_key)

In [5]:
googlemaps_client.geocode("Bloomington, IL")[0]["geometry"]['location']

{'lat': 40.4842027, 'lng': -88.99368729999999}

In [6]:
location1 = (googlemaps_client.geocode("Bloomington, IL")[0]["geometry"]['location']['lat'], 
             googlemaps_client.geocode("Bloomington, IL")[0]["geometry"]['location']['lng'])

In [7]:
location2 = (googlemaps_client.geocode("Cleveland, OH")[0]["geometry"]['location']['lat'], 
             googlemaps_client.geocode("Cleveland, OH")[0]["geometry"]['location']['lng'])

In [8]:
location1

(40.4842027, -88.99368729999999)

In [9]:
location2

(41.49932, -81.6943605)

In [17]:
#Create the map
fig = gmaps.figure()
#create the layer
layer = gmaps.directions.Directions(location1, location2, mode='car')
#Add the layer
fig.add_layer(layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
direction_route_info = googlemaps_client.directions(location1, location2, mode = "driving")[0]

In [21]:
direction_route_info.keys()

dict_keys(['bounds', 'copyrights', 'legs', 'overview_polyline', 'summary', 'warnings', 'waypoint_order'])

In [25]:
direction_route_info['legs'][0]['distance']['text']

'451 mi'

In [26]:
direction_route_info['legs'][0]['duration']['text']

{'text': '7 hours 3 mins', 'value': 25408}